In [1]:
%autosave 0

Autosave disabled


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn import linear_model
import pandas as pd
import numpy as np
import os
from sklearn import metrics

In [3]:
data = pd.read_csv(os.getcwd() + '\data\merged_train.csv')
data.head()

,State,County,FIPS,Total Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural,Democratic,Republican,Party
0,AZ,apache,4001,72346,18.571863,0.486551,5.947806,1.719515,50.598513,45.854643,13.322091,32460,15.807433,21.758252,88.941063,74.061076,16298,7810,1
1,AZ,cochise,4003,128177,56.299492,3.714395,34.403208,11.458374,49.069646,37.902276,19.756275,45383,8.567108,13.409171,76.837055,36.301067,17383,26929,0
2,AZ,coconino,4005,138064,54.619597,1.342855,13.711033,4.825298,50.581614,48.946141,10.873943,51106,8.238305,11.085381,65.791439,31.466066,34240,19249,1
3,AZ,gila,4007,53179,63.222325,0.552850,18.548675,4.249798,50.296170,32.238290,26.397638,40593,12.129932,15.729958,82.262624,41.062000,7643,12180,0
4,AZ,graham,4009,37529,51.461536,1.811932,32.097844,4.385942,46.313518,46.393456,12.315809,47422,14.424104,14.580797,86.675944,46.437399,3368,6870,0


**Patitioning using cross-validation method**

In [4]:
# Partitioning the dataset using KFold cross-validation
kf = KFold(n_splits=3, random_state=None, shuffle=False)
# Seperating democratic and republican counties
democratic = data[data['Party'] == 1]
republican = data[data['Party'] == 0]
democratic[democratic.columns[3:16]].head()

,Total Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural
0,72346,18.571863,0.486551,5.947806,1.719515,50.598513,45.854643,13.322091,32460,15.807433,21.758252,88.941063,74.061076
2,138064,54.619597,1.342855,13.711033,4.825298,50.581614,48.946141,10.873943,51106,8.238305,11.085381,65.791439,31.466066
6,4088549,56.918114,5.013612,30.286833,14.729333,50.549278,41.886620,13.837843,55676,6.808454,13.051927,69.031137,2.363800
9,1003338,53.271579,3.199719,36.105978,12.903428,50.807405,40.087388,17.801778,46764,9.214114,12.252238,69.199391,7.523491
10,46547,15.274883,0.199798,83.219112,32.644424,52.125808,43.300320,15.895761,38941,9.749896,25.206726,77.506775,26.883172


**Splitting, Standardization and Predicting Number of Votes for the Democratic Counties**

In [5]:
for train_indices, test_indices in kf.split(democratic):
    x_train, y_train = democratic[democratic.columns[3:16]].iloc[train_indices], democratic['Democratic'].iloc[train_indices]
    x_test, y_test = democratic[democratic.columns[3:16]].iloc[test_indices], democratic['Democratic'].iloc[test_indices]
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    model = linear_model.Lasso(alpha=1)
    fitted_model = model.fit(X = x_train, y = y_train)
    predicted = fitted_model.predict(x_test)
    corr_coef = np.corrcoef(predicted, y_test)[1, 0]
    R_squared = corr_coef ** 2
    print(fitted_model.coef_)
    print(model.score(X = x_test, y = y_test))
    print('\n')

[ 1.01969971e+05 -9.60713190e+03 -5.58120321e+03 -6.97142952e+03
 -1.03643045e+04 -1.63171036e+03 -1.19383626e+04 -6.56737147e+03
  6.79834927e+01  1.54459828e+03  6.55212689e+03 -1.87599484e+04
 -5.48863302e+03]
0.7941212111708189


[ 1.08666941e+05  2.28680581e+03 -4.00190852e+03 -2.03475132e+04
  1.50528543e+04  7.17168705e+01 -4.77052467e+03 -3.02049030e+02
  6.27731138e+03  4.82325731e+03  1.29902046e+04 -1.68962285e+04
 -2.57527990e+03]
0.757702665058086


[117433.62359741  -6574.78254091   3361.99107289  -7681.57584571
  -4489.43033916   1810.63410005  -2731.54492955   6675.1016213
   2808.41768644   2859.3911251    9256.20538912 -24613.44719151
  -2984.17563558]
0.8150182560002226




Evaluation Metrics 

In [31]:
print("Explained Variance Score: ", metrics.explained_variance_score(y_test, predicted))
print("Max Error: ", metrics.max_error(y_test, predicted))
print("Absolute Error: ", metrics.mean_absolute_error(y_test, predicted))

#cannot use because contain negative values 
#print("Mean squared logarithmic error regression loss: ", metrics.mean_squared_log_error(y_train, predicted))
# other metrics not used for the same reason‘neg_mean_squared_error’,‘neg_mean_squared_log_error’,‘neg_median_absolute_error’

Explained Variance Score:  0.9512162883788424
Max Error:  42586.5464536729
Absolute Error:  2431.6505595451026


**Splitting, Standardization and Predicting Number of Votes for the Democratic Counties**

In [6]:
for train_indices, test_indices in kf.split(republican):
    x_train, y_train = republican[republican.columns[3:16]].iloc[train_indices], republican['Republican'].iloc[train_indices]
    x_test, y_test = republican[republican.columns[3:16]].iloc[test_indices], republican['Republican'].iloc[test_indices]
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    model = linear_model.LinearRegression()
    fitted_model = model.fit(X = x_train, y = y_train)
    predicted = fitted_model.predict(x_test)
    corr_coef = np.corrcoef(predicted, y_test)[1, 0]
    R_squared = corr_coef ** 2
    print(fitted_model.coef_)
    print(model.score(X = x_test, y = y_test))
    print('\n')

[ 2.03123681e+04  6.70812988e+02 -1.99197588e+02  2.84889012e+02
 -3.64390951e+02  9.32786013e+00 -7.65390801e+02  3.21153436e+02
  1.37541623e+03  7.08687541e+01  1.02079117e+03 -1.28012577e+03
 -4.08437134e+02]
0.8848886457718538


[21414.24986791  1190.55094764   395.85523248   477.64669261
   513.05940452  -116.04140847 -1003.80465191  1048.25606177
  1475.39294803   635.18447266   419.73667663 -1051.81771342
  -545.52271291]
0.9415003960985364


[21637.65666314  1674.67218062   371.86754393   670.70356898
   704.48493661  -201.41158565  -924.07533003  1526.35049377
  1792.79222918   661.06673937   697.61622054  -714.61917245
  -525.07478444]
0.9510012082444844




Evaluation Metrics 

In [30]:
print("Explained Variance Score: ", metrics.explained_variance_score(y_test, predicted))
print("Max Error: ", metrics.max_error(y_test, predicted))
print("Absolute Error: ", metrics.mean_absolute_error(y_test, predicted))

#cannot use because contain negative values 
#print("Mean squared logarithmic error regression loss: ", metrics.mean_squared_log_error(y_train, predicted))
# other metrics not used for the same reason‘neg_mean_squared_error’,‘neg_mean_squared_log_error’,‘neg_median_absolute_error’

Explained Variance Score:  0.9512162883788424
Max Error:  42586.5464536729
Absolute Error:  2431.6505595451026
